In [8]:
%load_ext autoreload
%autoreload 2

from util_0803_2 import *
# from myalgorithm_0803_1 import algorithm

import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 실험 내용
- set을 이용해서 이미 불가능하다고 판단된 주문의 조합은 다시 확인하지 않기

In [46]:
problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_15.json'
# problem_file = '../alg_test_problems_20240429/TEST_K200_1.json'

testcase_name = problem_file.split('\\')[-1].split('.')[0]

start_time = time.time()

## ------------------- 기본 변수 할당 ---------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

inf = float('inf')

timelimit = 60

## ------------------- 초기 상태 할당 코드 -------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

inf = float('inf')

car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

init_availables = [rider.available_number for rider in ALL_RIDERS]

all_bundles = []
for ord in ALL_ORDERS:
    new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
    car_rider.available_number -= 1
    all_bundles.append(new_bundle)

## ------------------  주문 조합 가능성 행렬 제작하기 -----------------------------

order_comb_possibility = [[True] * K for _ in range(K)]

for i in range(len(all_bundles)):
    for j in range(i + 1, len(all_bundles)):
        bundle1 = all_bundles[i]
        bundle2 = all_bundles[j]

        order_num1 = bundle1.shop_seq[0]
        order_num2 = bundle2.shop_seq[0]

        ip = try_merging_bundles_by_dist(K, DIST, ALL_ORDERS, ALL_RIDERS, bundle1, bundle2)

        if not ip:
            order_comb_possibility[order_num1][order_num2] = False
            order_comb_possibility[order_num2][order_num1] = False

impossible_order_comb_st = set()

min_init_cost = inf
min_init_cost_bundles = []
min_init_cost_rider_availables = []

weight_grid = [(1, -1), (1, -1.5), (1, -2), (1, -2.5), (1, -3), (1, -3.5), (0.5, 1), (0.5, 2), (0.5, 3)]

temp_process_time = -1
for weight1, weight2 in weight_grid:
    cur_time = time.time()

    if cur_time - start_time > 30:
        break

    temp_start_time = time.time()

    bundles, result_rider_availables, cost = get_init_bundle_4_order_bundle_prefered_with_reassigning_riders(
            K, ALL_RIDERS, ALL_ORDERS, DIST, init_availables, weight1, weight2, order_comb_possibility, impossible_order_comb_st)
    temp_end_time = time.time()

    temp_process_time = temp_end_time - temp_start_time

    print(temp_process_time, cost)

    if cost < min_init_cost:
        min_init_cost = cost
        min_init_cost_bundles = bundles
        min_init_cost_rider_availables = result_rider_availables

0.33060336112976074 4412.215499999999
0.22144246101379395 4337.919499999999
0.13155817985534668 4370.759999999999
0.13666749000549316 4389.153499999999
0.09215426445007324 4457.306
0.13454985618591309 4476.911
0.31107449531555176 4551.579999999999
0.22564411163330078 4633.433999999999
0.20778274536132812 4731.247


In [ ]:
# 기본
0.34907984733581543 4412.215499999999
0.3235352039337158 4346.513999999999
0.33890724182128906 4390.8944999999985
0.3010122776031494 4406.860499999999
0.2786216735839844 4454.075499999999
0.3336360454559326 4468.93
0.355849027633667 4559.7435
0.3375818729400635 4629.9145
0.3975405693054199 4618.075999999999

# 불가능한 셋 확인
0.3295269012451172 4412.215499999999
0.21972107887268066 4337.919499999999
0.13394522666931152 4370.759999999999
0.1391456127166748 4389.153499999999
0.09472036361694336 4457.306
0.1281287670135498 4476.911
0.298184871673584 4551.579999999999
0.2136082649230957 4633.433999999999
0.19490742683410645 4731.247

# reduced 불가능한 셋 확인
0.3341975212097168 4412.215499999999
0.2240755558013916 4337.919499999999
0.13451886177062988 4370.759999999999
0.14487600326538086 4389.153499999999
0.09656453132629395 4457.306
0.1391897201538086 4476.911
0.29976511001586914 4551.579999999999
0.21927976608276367 4633.433999999999
0.20402121543884277 4731.247



> 확인 결과 차원 압축을 굳이 할 필요 없을 수도 있음

In [42]:
def reduce_dimension(sorted_orders):
    cur_w = 1

    reduced_value = 0
    for v in sorted_orders:
        to_add = (v + 1) * cur_w
        reduced_value += to_add

        cur_w *= 500

    return reduced_value

reduce_dimension((2, 1))

1003